In [16]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [17]:
def comment(url):
    wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver"
    driver = webdriver.Chrome(wd)
    driver.get(url)
    pages = 0
    try:
        while True:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#cbox_module > div > div.u_cbox_paginate > a"))).click()
            time.sleep(1.5)
            print(pages, end=" ")
            pages+=1
    except exceptions.ElementNotVisibleException as e: # 페이지 끝
        pass
    
    except Exception as e: # 다른 예외 발생시 확인
        print(e)
    html = driver.page_source
    dom = BeautifulSoup(html, "lxml")
    # 댓글이 들어있는 페이지 전체 크롤링
    comments_raw = dom.find_all("span", {"class" : "u_cbox_contents"})
    # 댓글의 text만 뽑는다.
    comments = [comment.text for comment in comments_raw]
    return comments

In [18]:
comment('https://entertain.naver.com/read?oid=109&aid=0004166734')

Message: 



['ㅋㅋㅋㅋㅋ이 4명 케미 왜이리 웃김 ㅋㅋㅋㅋ 수다만 떨었는데도 빵빵터짐 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '의미부여없이 웃기기로는 놀면뭐하니에서 오늘이 젤웃겼음ㅋㅋ지석진 조세호냅두고선 문닫고도망치는거 최고의 킬링포인트ㅋㅋㅋㅋ',
 '유느 행복해하는거 넘 보기좋네요ㅎㅎㅎ',
 '오프닝으로 뽕뽑넼ㅋㅋㅋㅋㅋ개웃겨',
 '유느가 기뻐하면 좋아요😆😆😆😆 토크만으로도 웃김',
 '유재석 방송 아닌 진짜웃는것같아 보는사람도 조았어요. 그리고 실제로도 책마니 읽는다하네요..괜히 1인자가 아님',
 '유재석 저렇게 즐거워 죽을 거 같은 얼굴로 미친듯이 웃는 거 근 5년 만에 처음 보는 듯. 나머지 셋 보고 웃는 건데 오히려 그 셋은 덤덤하고 혼자 웃겨서 뒤로 넘어가려는 게 너무 웃겨ㅋㅋㅋㅋㅋㅋㅋㅋ',
 '지석진 땀닦은 휴지보고 이광수가 살점떨어진줄 알았다고 한겈ㅋㅋㅋㅋㅋㅋㅋ진짜웃겨',
 '런닝맨에서 수년을봤는데도 광수나오니까 뭔가신선함',
 '소중한 우리 산슬이가 타고있어요 ㅋㅋㅋㅋㅋㅋㅋ']

In [22]:
from bs4 import BeautifulSoup
import urllib.request
import re

In [23]:
def get_text(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='articeBody'):
        text = text + str(item.find_all(text=True))
    return text

In [26]:
txt=get_text('https://entertain.naver.com/read?oid=109&aid=0004166734')

In [25]:
def clean_text(text):
    cleaned_text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\.]+", " ", text)
    return cleaned_text

In [27]:
clean_text(txt)

' 놀면 뭐하니 방송화면 캡처 김보라 기자 유재석 지석진 이광수 조세호가 포상 휴가를 즐겼다. 일 오후 방송된 예능프로그램 놀면 뭐하니 는 유재석의 포상 휴가기가 담긴 가운데 지석진 조세호 이광수가 함께 했다. 세 사람은 유재석과 단둘이 대 로 떠나는 여행인 줄 알고 왔는데 알고 보니 인이 떠나는 여행이 됐다. 이유인즉슨 제작진이 시간이 되는 사람을 알아 보기 위해 인에게 동시에 물어봤던 것. 지석진은 이게 이 프로그램의 위엄이다. 시청률이 두 자릿수가 나왔다 며 그래서 나왔다 라고 전해 웃음을 안겼다. 그러면서 지석진은 많이 보던 얼굴이라 캐스팅이 애매하다. 이게 뭐냐 런닝맨 이냐 라고 비하했다. 예능 런닝맨 에 동반 출연하는 이광수가 등장했기 때문. 이에 유재석은 내가 친한 사람들이 엄청나게 많지 않다 며 가수 유희열 이적과 친하다고 털어놨다. 이에 조세호는 우리가 만만한 거다 라고 맞아쳐 웃음을 안겼다. 그러면서 유재석 형은 자기보다 지식이 낮은 사람을 좋아한다 고 디스했다. 놀면 뭐하니 방송화면 캡처 이광수는 이날의 여행을 위해 제가 좋아하는 걸 하고 싶은 걸 솔직하게 적었다. 걷는 걸 좋아한다 며 여행하면서 차 타는 것도 좋지만 걸으며 보는 걸 좋아한다 고 말했다. 이광수의 의견에 따라 유재석은 남산 돈가스 집까지 걸어 가자고 했다. 이들은 아침 시에 연 남산 돈가스 집에서 아침식사를 했다. 이날 지석진이 이태원 가자. 스티커 사진을 찍자 는 등 다양한 의견을 내놓았는데 모두가 무시해 웃음을 유발했다. 식사를 마친 유재석은 쇼핑을 하러 가자 고 제안했고 네 사람은 이태원으로 발걸음을 옮겼다. 유재석이 운전하는 차에 몸을 싣은 지석진 조세호 이광수. 차 안에서도 이들의 수다가 시작됐다. 세 사람은 구경만 해도 재미있다 고 높은 기대감을 드러냈다. 우리 넷이 어디 가는 거 처음이다 라고 기뻐하는 유재석. 그는 가장 먼저 서점으로 세 명을 이끌었다. 일동 숙연해지는 순간. 조세호는 후드티 하나 살까 했는데 깜짝이야 라고 말문을 열었다. 놀면 뭐하니 방송화면